In [1]:
##################################################################################################
### This script is ML Classification template, which should be applicable to most MLC projects ###
##################################################################################################

"""Structure of the script:
1. Load all needed libraries and functions.
2. Load data, do preliminary data exploration.
3. [Optional] Transform skewed variables.
4. Trnasform features depending on their type. OHC.
5. Create subsamples.
6. Do scaling.
7. Fit models, selecting hyperparameters via CV grid search.
8. Evaluate performance of the selected models on test sample.
"""
# it is not the latest ML template. See HackerRank_ML_availability_final for later version.

'Structure of the script:\n1. Load all needed libraries and functions.\n2. Load data, do preliminary data exploration.\n3. [Optional] Transform skewed variables.\n4. Trnasform features depending on their type. OHC.\n5. Create subsamples.\n6. Do scaling.\n7. Fit models, selecting hyperparameters via CV grid search.\n8. Evaluate performance of the selected models on test sample.\n'

In [2]:
### 1.Load main libraries ###

import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import time

from sklearn import svm
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier

pd.set_option('display.max_columns', 20)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
def draw_histograms(df, variables, n_rows, n_cols):
    # stolen from https://stackoverflow.com/questions/29530355/plotting-multiple-histograms-in-grid
    fig=plt.figure()
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name+" Distribution")
    fig.tight_layout()  
    plt.show()

In [4]:
### 2.Load data ###

path = '../input/titanic/train.csv'
df = pd.read_csv(path) # titanic_fullsample
print(df.shape)
df.drop(columns=['Name', 'Ticket', 'Cabin'],inplace=True)
df.loc[df.Age.isnull(),'Age'] = df.Age.median()
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [ ]:
test_data=pd.read_csv('../input/hackerrank-availability/test (2).csv')
test_data.head()

In [ ]:
test_data=pd.read_csv('../input/hackerrank-availability/test (2).csv')
print(test_data.head())

print(data.shape, test_data.shape)
train = data.copy()
test = test_data.copy()
train['sample']='train'
test['yearly_availability'] = np.nan
test['sample']='test'
print(train.columns==test.columns)
# good, so we can concatenate train and test.

In [ ]:
print(df.describe())
# sns.pairplot(df[['Survived', 'Pclass', 'Age', 'Fare']])
draw_histograms(df, df.columns, 4, 3)

In [ ]:
#%% 3.Transform some skewed variables ###

df['Fare'] = np.log1p(df.Fare)

In [ ]:
#%% 4.Transform features depending on their type ###

# this is very important for ML application, where there are hundreds of features.
# If there are less than 20 features, can use standard approach.
# my approach of tackling one feature a time is not scalable. 

# use intuition to trim range or ordinary variables 
# can skip this step in general, since it is not scalable when number of features grows.
df.loc[df.SibSp>2,'SibSp']=3
df.loc[df.Parch>2,'Parch']=3

# identify binary and categorical variables
df_uniques = pd.DataFrame([[i, len(df[i].unique())] for i in df.columns], columns=['Variable', 'Unique Values']).set_index('Variable')
print(df_uniques)

binary_variables = list(df_uniques[df_uniques['Unique Values'] == 2].index)
categorical_variables = list(df_uniques[(6 >= df_uniques['Unique Values']) & (df_uniques['Unique Values'] > 2)].index)
numeric_variables = list(set(df.columns) - set(categorical_variables) - set(binary_variables))
print('Binary variables are ', binary_variables)
print('Categorical variables are ', categorical_variables)
print('Numeric variables are ', numeric_variables)

In [ ]:
# ohc for binary variables #
lb = LabelBinarizer()
binary_variables.remove('Survived')
for column in binary_variables:
    df[column] = lb.fit_transform(df[column])

# ohc for categorical variables #
df = pd.get_dummies(df, columns = categorical_variables, drop_first=True)

print(df.shape)
print(df.head())

In [ ]:
# %% 5.Creating subsamples ###

y = df['Survived']
X = df.drop(columns=['Survived'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
# %% 6.scaling numeric variables ###

draw_histograms(X_train, numeric_variables, 2, 3)

In [ ]:
ss = StandardScaler()

for column in [numeric_variables]:
    X_train[column] = ss.fit_transform(X_train[column])
    X_test[column] = ss.transform(X_test[column])

In [ ]:
####################
### 7.Fit models ###
####################

#%% Logistic regression ###

grid_values = {'penalty': ['l2'], 'C': list(np.arange(1,10.5,0.5))}
lr = LogisticRegression()
model_lr = GridSearchCV(lr, param_grid=grid_values, cv = 8)
model_lr.fit(X_train, y_train)
print(model_lr.best_score_, model_lr.best_params_)

# model_lr.predict(X_test)

In [ ]:
#%% KNN ###

grid_values = dict(n_neighbors=np.arange(1,40))
knnm = KNeighborsClassifier()   
model_knn = GridSearchCV(knnm, param_grid=grid_values, cv = 8)
model_knn.fit(X_train, y_train)
print(model_knn.best_score_, model_knn.best_params_)

In [ ]:
#%% SVM ###

grid_values = {'C': np.arange(0.1, 3, 0.1)} 
svmm = svm.SVC(kernel='rbf')
model_svm = GridSearchCV(svmm, param_grid=grid_values, cv = 8)
model_svm.fit(X_train, y_train)
print(model_svm.best_score_, model_svm.best_params_)

In [ ]:
#%% RF ###

# may look here: https://www.geeksforgeeks.org/hyperparameter-tuning/

grid_values = [{'max_depth': list(range(2, 11, 1)), 'max_features': list(np.arange(0.3,0.71,0.1))}]
rfc = RandomForestClassifier(random_state=42)
model_rf = GridSearchCV(rfc, grid_values, cv = 8, scoring='accuracy')
model_rf.fit(X_train, y_train)
print(model_rf.best_score_, model_rf.best_params_)

In [ ]:
#%% XGBoost ###
# run this code only on Kaggle with GPU

estimator = XGBClassifier(
    nthread=4,
    seed=42,
    use_label_encoder=False
)

parameters = {
    'max_depth': range (2, 4, 1),
    'n_estimators': range(100, 301, 50),
    'learning_rate': [0.001, 0.025, 0.05, 0.075]
}
t1 = time.time()
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = -1,
    cv = 8,
    verbose=True
)

grid_search.fit(X_train, y_train, eval_metric='rmse')
print(grid_search.best_score_, grid_search.best_params_)
print(time.time()-t1)

In [ ]:
param['gpu_id'] = 0
param['tree_method'] = 'gpu_hist'

In [ ]:
xgbc = XGBClassifier(nthread=4, seed=42, use_label_encoder=False,
                     max_depth=2, n_estimators=100, learning_rate=0.05)
xgbc.fit(X_train, y_train)

In [ ]:
# here I will add ANN

from keras.models  import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
X_train.shape

In [ ]:
model_2 = Sequential()
model_2.add(Dense(50, input_shape=(X_train.shape[1],), activation="relu"))
model_2.add(Dropout(0.4))
model_2.add(BatchNormalization())
model_2.add(Dense(10, activation="relu"))
model_2.add(Dropout(0.4))
model_2.add(BatchNormalization())
model_2.add(Dense(1, activation="sigmoid"))

es = EarlyStopping(monitor='val_loss', patience=20)

model_2.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
run_hist_2 = model_2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300, callbacks=[es])

In [ ]:
# played a lot with ann. the best acc i can reliable achieve is 78.5%.
# but the second time (test set) I get 83%...


In [ ]:
#%% 8.Evaluate performance oos ###

yhat_lm = model_lr.predict(X_test)
yhat_knn = model_knn.predict(X_test)
yhat_svm = model_svm.predict(X_test)
yhat_rf = model_rf.predict(X_test)
yhat_bt = xgbc.predict(X_test)
print('Accuracy of logistic regression is ', 1-(np.abs(yhat_lm-y_test)).mean())
print('Accuracy of KNN is ', 1-(np.abs(yhat_knn-y_test)).mean())
print('Accuracy of SVM is ', 1-(np.abs(yhat_svm-y_test)).mean())
print('Accuracy of RF is ', 1-(np.abs(yhat_rf-y_test)).mean())
print('Accuracy of Boosted Tree is ', 1-(np.abs(yhat_bt-y_test)).mean())

In [ ]:
### Export results ###
yhat_bt = boosted_tree.predict(X_test)
yhat_bt[0:20]

submissions_df = pd.DataFrame({'id': test.id, 'yearly_availability': yhat_bt}, columns=['id', 'yearly_availability'])
submissions_df

submissions_df.to_csv('submissions.csv',index=False)
